In [ ]:
import multiprocessing as mp
import parmap
import sys, os
import pandas as pd
import gzip

## List of inputs
ref = pd.read_csv(snakemake.input.ref, compression="gzip", sep="\t")
# ref = pd.read_csv("/scratch/tweber/DATA/1000G_SNV_with_GT/OTF/OTF/BCFTOOLS_CONCAT_TAB/merge.txt.gz", compression="gzip", sep="\t")
ref

In [ ]:
ref.ID.nunique()

In [ ]:
ref.SAMPLE.nunique()

In [ ]:
m = mp.Manager()
l_df_gb = m.list()
l_df = m.list()

def mp_vcf(vcf, l_df_gb, l_df):

    # Nb of rows to skip at the beginning of the VCF
    skip = len([e.decode("ISO-8859–1").split('\n') for e in gzip.open("{}".format(vcf, "rb")) if e.decode("ISO-8859–1").split('\n')[0].startswith("##")])
    # chrom = vcf.split("/")[-3]
    sample = vcf.replace(".vcf.gz", "")
    df = pd.read_csv(vcf, compression="gzip", skiprows=skip, sep="\t")
    df["#CHROM"] = df["#CHROM"].astype(str)
    df["#CHROM"] = df["#CHROM"].str.replace("chr", "")
    df["ID"] = df["#CHROM"].astype(str) + ":" + df["POS"].astype(str) + ":" + df["REF"] + ":" + df["ALT"]

    # df = df.loc[df["QUAL"] >= 10]

    merge_df = pd.merge(ref, df, on="ID", how='right')

    merge_df["SAMPLE"] = merge_df["SAMPLE"].fillna("NAN")
    merge_df["QUERY_SAMPLE_CELL"] = sample
    

    merge_df_gb = merge_df.groupby("SAMPLE")["ID"].nunique().sort_values(ascending=False).reset_index()
    merge_df_gb["QUERY_SAMPLE_CELL"] = sample
    merge_df_gb["Rank"] = list(range(1,1+merge_df_gb.shape[0]))
    merge_df_gb["Total_SNP"] = df.shape[0]

    l_df_gb.append(merge_df_gb)
    l_df.append(merge_df)
# f = snakemake.input.gt_sample_folder
# f = "/scratch/tweber/DATA/1000G_SNV_with_GT/OTF/GENOTYPING_OTF_POOL1/"
# f_ldir = sorted([e for e in os.listdir(f) if e.endswith(".vcf.gz")])
f_ldir = sorted(list(snakemake.input.vcf))
parmap.starmap(mp_vcf, list(zip(f_ldir)), l_df_gb, l_df)

In [ ]:
pd.options.display.max_rows = 250
concat_df = pd.concat(list(l_df))
concat_df

In [ ]:
# concat_counts_df = concat_df.loc[concat_df["QUAL"] >= 10].groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
concat_counts_df = concat_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
concat_counts_df["Rank"] = concat_counts_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].transform(lambda r: range(1, len(r) + 1))
concat_counts_df

In [ ]:
coverage_df = pd.read_csv(snakemake.input.coverage, sep="\t")
# coverage_df = pd.read_csv("/scratch/tweber/DATA/1000G_SNV_with_GT/OTF/COVERAGE/HGSVCxPool1/MERGE/merge_coverage.txt", sep="\t")
# coverage_df.describe()

In [ ]:
ashleys_predictions = pd.read_csv(snakemake.input.ashleys_predictions, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL", "prediction" : "ashleys_prediction", "probability": "ashleys_probability"}, axis=1).sort_values(by="QUERY_SAMPLE_CELL")
# ashleys_predictions = pd.read_csv("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/cell_selection/labels.tsv", sep="\t").rename({"cell": "QUERY_SAMPLE_CELL", "prediction" : "ashleys_prediction", "probability": "ashleys_probability"}, axis=1).sort_values(by="QUERY_SAMPLE_CELL")
ashleys_predictions["QUERY_SAMPLE_CELL"] = ashleys_predictions["QUERY_SAMPLE_CELL"].str.replace(".sort.mdup.bam", "")
# mc_predictions = pd.read_csv("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/counts/HGSVCxPool1.info_raw", skiprows=13, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL"}, axis=1)[["QUERY_SAMPLE_CELL", "mapped", "good", "pass1"]].rename({"mapped": "reads_mapped", "good" : "reads_used", "pass1": "mc_coverage_compliant"}, axis=1)
mc_predictions = pd.read_csv(snakemake.input.mc_predictions, skiprows=13, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL"}, axis=1)[["QUERY_SAMPLE_CELL", "mapped", "good", "pass1"]].rename({"mapped": "reads_mapped", "good" : "reads_used", "pass1": "mc_coverage_compliant"}, axis=1)

ashleys_mc_predictions = pd.merge(
    ashleys_predictions,
    mc_predictions,
    on="QUERY_SAMPLE_CELL"
)
ashleys_mc_predictions.loc[(ashleys_mc_predictions["ashleys_prediction"] == 1) & (ashleys_mc_predictions["mc_coverage_compliant"] == 1), "Used/Not used in MC"] = 1
ashleys_mc_predictions["Used/Not used in MC"] = ashleys_mc_predictions["Used/Not used in MC"].fillna(0)
ashleys_mc_predictions["Used/Not used in MC"] = ashleys_mc_predictions["Used/Not used in MC"].astype(int)
ashleys_mc_predictions

In [ ]:
concat_df

In [ ]:
qual_cutoff = 10
merge_df = pd.merge(
        concat_df.groupby("QUERY_SAMPLE_CELL")["ID"].nunique().rename("QUAL >= 0").reset_index(),
        concat_df.loc[concat_df["QUAL"] >= qual_cutoff].groupby("QUERY_SAMPLE_CELL")["ID"].nunique().rename("QUAL >= {}".format(str(qual_cutoff))).reset_index(),
        on="QUERY_SAMPLE_CELL",how="outer").sort_values(by="QUERY_SAMPLE_CELL")

import numpy as np
vcf_list = list(concat_df.QUERY_SAMPLE_CELL.unique())
empty_samples_list = [{"QUERY_SAMPLE_CELL" : "HGSVCPool1x02PE20{}".format(str(e)), "QUAL >= 0" : np.nan, "QUAL >= {}".format(str(qual_cutoff)) : np.nan, } for e in list(range(301, 397)) if "HGSVCPool1x02PE20{}".format(str(e)) not in vcf_list]

merge_df = pd.concat([merge_df, pd.DataFrame(empty_samples_list)]).sort_values(by="QUERY_SAMPLE_CELL")

concat_counts_df = concat_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
concat_counts_df["Rank"] = concat_counts_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].transform(lambda r: range(1, len(r) + 1))

merge_df = pd.merge(
        merge_df,
        pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"], values=["SAMPLE"], aggfunc=lambda x: ' '.join(x)),
        on="QUERY_SAMPLE_CELL",
        how="left"
)
merge_df = pd.merge(
        merge_df,
        pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"]),
        on="QUERY_SAMPLE_CELL",
        how="left"
)

concat_counts_df = concat_df.loc[concat_df["QUAL"] >= qual_cutoff].groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
concat_counts_df["Rank"] = concat_counts_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].transform(lambda r: range(1, len(r) + 1))

merge_df = pd.merge(
        merge_df,
        pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"], values=["SAMPLE"], aggfunc=lambda x: ' '.join(x)),
        on="QUERY_SAMPLE_CELL",
        how="left"
)
merge_df = pd.merge(
        merge_df,
        pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"]),
        on="QUERY_SAMPLE_CELL",
        how="left"
)
merge_df = pd.merge(
        merge_df,
        coverage_df,
        on="QUERY_SAMPLE_CELL",
        how="left"
)
merge_df = pd.merge(
        merge_df,
        ashleys_mc_predictions,
        on="QUERY_SAMPLE_CELL",
        how="left"
)

pd.options.display.max_columns = 50
merge_df.columns = ['QUERY_SAMPLE_CELL', 'QUAL>=0', 'QUAL>={}'.format(str(qual_cutoff)), 'QUAL>=0_SAMPLE_1', 'QUAL>=0_SAMPLE_2', 'QUAL>=0_SAMPLE_3', 'QUAL>=0_Counts_1', 'QUAL>=0_Counts_2', 'QUAL>=0_Counts_3',
'QUAL>={}_SAMPLE_1'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_2'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_3'.format(str(qual_cutoff)), 'QUAL>={}_Counts_1'.format(str(qual_cutoff)), 'QUAL>={}_Counts_2'.format(str(qual_cutoff)), 'QUAL>={}_Counts_3'.format(str(qual_cutoff)),  'Average_Coverage', 'ashleys_prediction', 'ashleys_probability', 'reads_mapped', 'reads_used', 'mc_coverage_compliant', 'Used/Not used in MC']


# merge_df = merge_df[
#         [
#              'QUERY_SAMPLE_CELL',   'QUAL>={}'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_1'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_2'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_3'.format(str(qual_cutoff)), 'QUAL>={}_Counts_1'.format(str(qual_cutoff)), 'QUAL>={}_Counts_2'.format(str(qual_cutoff)), 'QUAL>={}_Counts_3'.format(str(qual_cutoff)),  'Average_Coverage', 'ashleys_prediction', 'ashleys_probability', 'reads_mapped', 'reads_used', 'mc_coverage_compliant', 'Used/Not used in MC'
#         ]
# ]
merge_df["Average_Coverage"] = merge_df["Average_Coverage"]*100

# merge_df = merge_df.loc[merge_df["mc_coverage_compliant"] == 1].sort_values(by="Average_Coverage")

# merge_df.to_excel("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/config/stats_pooling.xlsx", index=False)
merge_df.to_excel(snakemake.output.stats, index=False)

# import os, sys, stat
# os.chmod("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/config/stats_pooling.xlsx", stat.S_IRWXG)

merge_df.sort_values(by="Average_Coverage")


In [ ]:
# merge_df[['ashleys_prediction', 'mc_coverage_compliant', 'Used/Not used in MC']].sum()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# from matplotlib.colors import LogNorm, Normalize


# plt.style.use('default')
# plt.figure(figsize=(10,20))


# merge_df

# ax = sns.barplot(data=merge_df.sort_values(by="QUERY_SAMPLE_CELL"),  y="QUERY_SAMPLE_CELL", x="Average_Coverage", hue="mc_coverage_compliant")
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# ax.axvline(0.34, ls="--", color="grey", lw=0.5)
# ax.set_xlabel("Average Coverage %")

In [ ]:
merge_df.columns